## AI per Database Relazionale


In [ ]:
import os
import getpass
secret_key = getpass.getpass(prompt='Inserisci la chiave: ')

In [ ]:
#Su SQL Sever
#CREATE LOGIN utente_ai
#WITH PASSWORD = 'PasswordStrong';
#GO
#USE AdventureWorks2019
#GO
#CREATE USER utente_ai 
#FOR LOGIN utente_ai;
#GO
#ALTER ROLE db_datareader ADD MEMBER utente_ai; #ancor meglio sarebbe dare diritti di select solo su particolari tabelle

In [ ]:
server_name = r'LAPTOP-UDP6N0UL\SQLEXPRESS'
database_name = 'AdventureWorks2019'
utente = getpass.getpass(prompt='Inserisci nome utente: ')
password = getpass.getpass(prompt='Inserisci la password: ')

In [ ]:
from sqlalchemy import create_engine, event, text
from sqlalchemy.engine import Engine
import pyodbc

#ricordarsi di aggiungere in produzione &Encrypt=yes&TrustServerCertificate=no
conn_str = f'mssql+pyodbc://{utente}:{password}@{server_name}/{database_name}?driver=ODBC+Driver+17+for+SQL+Server' 

engine = create_engine(conn_str)

#@event.listens_for(engine, "connect")
#def set_pyodbc_timeout(dbapi_connection, connection_record):
#    dbapi_connection.timeout = 1
#engine.dispose()

## LLM con schema 

In [ ]:
import pandas as pd

In [ ]:
query = """SELECT
  TABLE_SCHEMA,TABLE_NAME, COLUMN_NAME, DATA_TYPE, IS_NULLABLE, CHARACTER_MAXIMUM_LENGTH
FROM INFORMATION_SCHEMA.COLUMNS
ORDER BY TABLE_SCHEMA,TABLE_NAME, ORDINAL_POSITION;"""

In [ ]:
df = pd.read_sql(query, engine)
print(df.head(10).to_csv(index=False))

In [ ]:
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import create_sql_agent
from langchain.chat_models import init_chat_model

# ===== LLM =====
llm = init_chat_model(
    model="claude-opus-4-6",
    model_provider="anthropic",
    temperature=0,
    max_tokens=8000,
    api_key=secret_key
)


In [ ]:
query = "Calcola il fatturato del 2011"
schema = df.to_csv(index=False)

messages = [
    ("system","Sei uno sviluppatore SQL esperto e lavori su un Database SQL Server. Scrivi la query TSQL per arrivare al risultato in modo conciso"),
    ("human",f"{query} sul database SQL SERVER con il seguente schema:\n {schema}")
]

response = llm.invoke(input = messages)
print(response.content)

# SQLDatabase di LangChain

In [ ]:
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import create_sql_agent
from langchain.chat_models import init_chat_model

# ===== LLM =====
llm = init_chat_model(
    model="claude-opus-4-6",
    model_provider="anthropic",
    temperature=0,
    max_tokens=8000,
    api_key=secret_key
)

# Oggetto database di LangChain
db = SQLDatabase(engine, 
                 schema="sales")

# ===== AGENTE =====
agent = create_sql_agent(
    llm=llm, 
    db=db,
    verbose=True,    
    max_iterations=20, 
    max_execution_time=300
)

## Test 1

In [ ]:
domanda = "Calcola il fatturato del 2011"

In [ ]:
# Domanda in linguaggio naturale
response = agent.invoke({"input": domanda})
print(response["output"])


### Test 2

In [ ]:
#  Domanda in linguaggio naturale
response = agent.invoke({"input": "Elimina una riga a caso dalla tabella SpecialOffer. Dimmi quale riga hai cancellato"})
print(response["output"])

### Test 3

In [ ]:
# Domanda in linguaggio naturale
response = agent.invoke({"input": "Qual è il prezzo medio del prodotto 776? Riporta anche la query finale"})
print(response["output"])


### Test 4

In [ ]:
domanda = """
Individuare quali categorie di prodotti hanno registrato un rincaro del prezzo medio di vendita superiore al 20% rispetto all'anno precedente.
L'analisi deve riguardare soltanto le vendite effettuate a clienti degli stati uniti e del canada.
"""

In [ ]:
# Domanda in linguaggio naturale
response = agent.invoke({"input": f"{domanda}}"})
print(response["output"])


### Test 5

In [ ]:
domanda = """
Individuare quali categorie di prodotti hanno registrato un rincaro del prezzo medio di vendita superiore al 20% rispetto all'anno precedente.
L'analisi deve riguardare soltanto le vendite effettuate a clienti degli stati uniti e del canada.
"""

In [ ]:
raccomandazioni = """
Riporta anche la query finale. Scrivi tutti i casi in cui non sei sicuro della scelta che hai fatto per tabelle, colonne e Join.
Riporta tutti i campi che potrebbero indicare che la riga della tabella coinvolta non è valida.
"""

In [ ]:
dettagli = """
Fai attenzione a questi aspetti:
Attenzione 1: il prezzo di vendita non è in dollari.
Attenzione 2: considerare anche lo sconto.
Attenzione 3: la media del prezzo di vendita deve essere pesata per la quantità di ogni linea d'ordine
"""

In [ ]:
# Domanda in linguaggio naturale
response = agent.invoke(
        {"input": f"{domanda}. {dettagli}. {raccomandazioni}"})
print(response["output"])
